## Find Artist and Create Cover Art

In [1]:
# Find existing Asset and create an Attachment
#
# Main function, establishes a connection to RKVST using an App Registration then uses that
# to find existing Asset and create an attachment.
#
# Note: The purpose of RKVST Jupyter Notebooks is to provide simplified examples that one can easily execute and digest.
# The RKVST Python SDK is authored to work cleanly with more advanced coding techniques.
#
# RKVST Python SDK: https://github.com/rkvst/rkvst-python
#

In [2]:
from json import dumps as json_dumps
from os import getenv
from warnings import filterwarnings

from dotenv import load_dotenv

from archivist.archivist import Archivist
from archivist.proof_mechanism import ProofMechanism
from archivist.logger import set_logger

In [3]:
%reload_ext dotenv
%dotenv -o notebooks.env

In [4]:
# RKVST_URL, RKVST_APPREG_CLIENT, RKVST_APPREG_SECRET are environment variables that represent connection parameters.
#
# RKVST_URL = represents the url to the RKVST application
# RKVST_APPREG_CLIENT = represents the client ID from an Application Registration
# RKVST_APPREG_SECRET = represents the client secret from an Application Registration
# RKVST_UNIQUE_ID = is an environment variable that is a unique identifier
# RKVST_ARTIST_ATTACHMENT = represents the location of the attachment
RKVST_URL = getenv("RKVST_URL")
RKVST_APPREG_CLIENT = getenv("RKVST_APPREG_CLIENT")
RKVST_APPREG_SECRET = getenv("RKVST_APPREG_SECRET")
RKVST_UNIQUE_ID = getenv("RKVST_UNIQUE_ID")
if not RKVST_UNIQUE_ID:
    raise Exception("RKVST_UNIQUE_ID is undefined")
RKVST_ARTIST_ATTACHMENT = getenv("RKVST_ARTIST_ATTACHMENT")
if not RKVST_ARTIST_ATTACHMENT:
    raise Exception("RKVST_ARTIST_ATTACHMENT is undefined")

In [5]:
"""
Main function of Asset and Event creation.

* Connect to RKVST with client ID and client secret
* Creates an Asset and two Events
* Prints response of Asset and Event creation
"""

# Optional call to set the logger level.  The argument can be either
# "INFO" or "DEBUG".  For more sophisticated logging control see our
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
print("RKVST_URL", RKVST_URL)
arch = Archivist(RKVST_URL, (RKVST_APPREG_CLIENT, RKVST_APPREG_SECRET), max_time=300)

Connecting to RKVST
RKVST_URL https://app.rkvst.io


In [6]:
def upload_attachment(arch, path, name):
    """
    Obtains Attachments from "test_files" folder and creates json payload
    for upload
    """
    with open(RKVST_ARTIST_ATTACHMENT, "rb") as fd:
        blob = arch.attachments.upload(fd)
        attachment = {
            "arc_display_name": name,
            "arc_attribute_type": "arc_attachment",
            "arc_file_name": path,
            "arc_blob_identity": blob["identity"],
            "arc_blob_hash_value": blob["hash"]["value"],
            "arc_blob_hash_alg": blob["hash"]["alg"],
        }
        return attachment

In [7]:
def get_artist(arch, name, artistid):
    """
    Finds existing Artist asset by name and unique id
    """
    attrs = {
        "arc_display_type": "Artists",
        "arc_display_name": name,
        "artistid": artistid,
    }

    return arch.assets.read_by_signature(attrs=attrs)

In [8]:
def create_image(arch, asset):
    """
    Creates an Event that uploads a primary image for an Artist asset
    """
    attachments = upload_attachment(
        arch, "pexels-andrea-turner-707697.jpeg", "arc_primary_image"
    )

    props = {"operation": "Record", "behaviour": "RecordEvidence"}
    attrs = {
        "arc_description": "Attaching an image",
        "arc_display_type": "Primary image",
    }

    asset_attrs = {"arc_primary_image": attachments}

    return arch.events.create(
        asset["identity"], props=props, attrs=attrs, asset_attrs=asset_attrs
    )

In [9]:
# Finding existing artist by name and artist id
print("Finding Asset")
asset = get_artist(arch, "Adele Laurie Blue Adkins", RKVST_UNIQUE_ID)
print("Asset", json_dumps(asset, indent=4))

Finding Asset


Refresh token


Asset {
    "identity": "assets/17a59a16-3d5c-4d58-8ea6-1b86465ff32f",
    "behaviours": [
        "RecordEvidence",
        "Builtin",
        "AssetCreator",
    ],
    "attributes": {
        "arc_display_name": "Adele Laurie Blue Adkins",
        "arc_display_type": "Artists",
        "artistid": "989130159",
        "genre": "Soul",
        "stage_name": "Adele",
        "arc_description": "British Soul Singer"
    },
    "confirmation_status": "CONFIRMED",
    "tracked": "TRACKED",
    "owner": "0x5284e740A744F075E402f7fB0c4485532ddf4Af8",
    "at_time": "2023-06-02T21:01:56Z",
    "storage_integrity": "TENANT_STORAGE",
    "proof_mechanism": "SIMPLE_HASH",
    "chain_id": "8275868384",
    "public": false,
    "tenant_identity": "tenant/0a62f7c9-fd7b-4791-8041-01218d839ec1"
}


In [10]:
# Create an Event for Attachment upload
print("Creating Events for existing Asset")
image_event = create_image(arch, asset)
print("Event for Image", json_dumps(image_event, indent=4))

Creating Events for existing Asset
Event for Image {
    "identity": "assets/17a59a16-3d5c-4d58-8ea6-1b86465ff32f/events/11065c00-fd3b-4a76-9935-0703bcf30d68",
    "asset_identity": "assets/17a59a16-3d5c-4d58-8ea6-1b86465ff32f",
    "event_attributes": {
        "arc_display_type": "Primary image",
        "arc_description": "Attaching an image"
    },
    "asset_attributes": {
        "arc_primary_image": {
            "arc_blob_hash_value": "cf0dd630dcfb6e2eac65c362bf7d5ff382a4241ebf45a69a2541ee43320d4af6",
            "arc_blob_identity": "blobs/a39ed534-d1b3-4b23-84e0-01a2cceea9a2",
            "arc_display_name": "arc_primary_image",
            "arc_file_name": "pexels-andrea-turner-707697.jpeg",
            "arc_attribute_type": "arc_attachment",
            "arc_blob_hash_alg": "SHA256"
        }
    },
    "operation": "Record",
    "behaviour": "RecordEvidence",
    "timestamp_declared": "2023-06-02T21:02:01Z",
    "timestamp_accepted": "2023-06-02T21:02:01Z",
    "timestamp_